# мастер-класс по машинному обучению

ссылка на ноутбук:

# https://cutt.ly/6ebXD8O

контакты:
* vk: https://vk.com/de_evjeny
* e-mail: evjeny@ai-tonometry.com


# загрузка данных

In [0]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

In [0]:
images, target = datasets.load_digits(return_X_y=True)

In [0]:
images.shape

In [0]:
image_num = 0
image = images[image_num]

image = image.reshape(8, 8)
plt.imshow(image)
print("label:", target[image_num])

посмотрим яркость пикселей:

In [0]:
print("минимальное значение:", images.min(), ", максимальное значение:", images.max())

нормализуем картинки (переведем все значения в промежуток от 0 до 1):

In [0]:
images = images / 16.0

In [0]:
print("минимальное значение:", images.min(), ", максимальное значение:", images.max())

посмотрим метки классов:

In [0]:
target

In [0]:
x_train, x_test, y_train, y_test = train_test_split(images, target,
                                                    test_size=0.2,
                                                    random_state=42)

# sklearn-модельки

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

функция, которая понадобится позже:

In [0]:
def plot_confusion_matrix(y_true, y_pred, classes=np.arange(10),
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots(figsize=(6, 6))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           title=title,
           xticklabels=classes, yticklabels=classes,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

## логистическая регрессия

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
log_reg = LogisticRegression()

In [0]:
log_reg.fit(x_train, y_train)

In [0]:
print("точность на тренировочной выборке:", log_reg.score(x_train, y_train))
print("точность на тестовой выборке:", log_reg.score(x_test, y_test))

In [0]:
y_predicted = log_reg.predict(x_test)
plot_confusion_matrix(y_test, y_predicted, title="logistic regression")

## случайный лес

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
random_forest = RandomForestClassifier()

In [0]:
random_forest.fit(x_train, y_train)

In [0]:
print("точность на тренировочной выборке:", random_forest.score(x_train, y_train))
print("точность на тестовой выборке:", random_forest.score(x_test, y_test))

In [0]:
y_predicted = random_forest.predict(x_test)
plot_confusion_matrix(y_test, y_predicted, title="random forest")

In [0]:
feature_importances = random_forest.feature_importances_
features_square = feature_importances.reshape(8, 8)
plt.imshow(features_square, cmap="hot")

## перцептрон

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

In [0]:
from sklearn.neural_network import MLPClassifier

In [0]:
perceptron = MLPClassifier(hidden_layer_sizes=(40, 20))

In [0]:
perceptron.fit(x_train, y_train)

In [0]:
print("точность на тренировочной выборке:", perceptron.score(x_train, y_train))
print("точность на тестовой выборке:", perceptron.score(x_test, y_test))

In [0]:
y_predicted = perceptron.predict(x_test)
plot_confusion_matrix(y_test, y_predicted, title="perceptron")

# подбор параметров

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
log_reg_parameters = {
    "penalty": ["l1", "l2"],
    "tol": [1e-3, 1e-4, 1e-5],
    "solver": ["liblinear", "saga"]
}

In [0]:
grid_search_log_reg =  GridSearchCV(LogisticRegression(), log_reg_parameters,
                                    cv=2, n_jobs=-1)

In [0]:
grid_search_log_reg.fit(images, target)

In [0]:
print(grid_search_log_reg.best_params_)

In [0]:
best_log_reg = LogisticRegression()

In [0]:
best_log_reg.fit(x_train, y_train)

In [0]:
print("точность на тренировочной выборке:", best_log_reg.score(x_train, y_train))
print("точность на тестовой выборке:", best_log_reg.score(x_test, y_test))

# экспортирование модели

In [0]:
from pickle import dump

In [0]:
with open("model.pickle", "wb+") as f:
  dump(log_reg, f)

# развертывание модели

1. создать аккаунт на https://www.pythonanywhere.com

2. выбрать вкладку "Consoles" и нажать на "**Bash**"

3. установить библиотеки, введя в Bash-консоль команду:
```
python3 -m pip install scikit-learn joblib Flask Pillow numpy --user
```

4. выбрать вкладку "Web", нажать на "Add a new web app", затем - "Next"

![](https://github.com/evjeny/mk_senej/blob/master/images/step_4.png?raw=true)

5. выбрать "Flask" и "Python 3.7"

![](https://github.com/evjeny/mk_senej/blob/master/images/step_5_1.png?raw=true)

![](https://github.com/evjeny/mk_senej/blob/master/images/step_5_2.png?raw=true)

6. указать путь /home/<*username*>/mk_senej/app.py

![](https://github.com/evjeny/mk_senej/blob/master/images/step_6.png?raw=true)

7. в настройках веб-приложения в разделе код указать пути к папке "mk_senej"

![](https://github.com/evjeny/mk_senej/blob/master/images/step_7.png?raw=true)

8. снова зайти в Bash-консоль и выполнить следующие команды:
```
cd ~
rm -rf mk_senej
git clone https://github.com/evjeny/mk_senej.git
```
9. после - перейти обратно во вкладку с настройками приложения и нажать "Reload <*username*>.pythonanywhere.com"

![](https://github.com/evjeny/mk_senej/blob/master/images/step_9.png?raw=true)

10. перейти на http://<*username*>.pythonanywhere.com